In [60]:
import pandas as pd 
import numpy as np

In [61]:

rating= pd.read_csv('colla.csv')
rating


,userid,name,rate,id
0,a,cake,2,2
1,b,bun,5,5
2,c,cup,4,8
3,a,soup,2,4
4,a,ball,1,3
5,b,ladder,3,6
6,b,jump,4,1
7,c,bat,0,9
8,c,king,5,7
9,c,win,2,12


In [68]:
input= pd.read_csv('input.csv')

input

,userid,name,rate,id
0,k,ladder,5,6
1,k,jump,3,1


In [69]:
usersubset=rating[rating['id'].isin(input['id'].tolist())]
usersubset.head()

,userid,name,rate,id
5,b,ladder,3,6
6,b,jump,4,1


In [70]:
usersubsetGroup=usersubset.groupby(['userid'])

In [71]:
usersubsetGroup=sorted(usersubsetGroup, key=lambda x: len(x[1]), reverse=True)

In [72]:
usersubsetGroup[0]

('b',
   userid    name  rate  id
 5      b  ladder     3   6
 6      b    jump     4   1)

In [73]:
usersubsetGroup[0][0]

'b'

In [74]:
usersubsetGroup =usersubsetGroup[0:100]

In [75]:
pearsonCorrelationDict={}

for name,group in usersubsetGroup:
    group=group.sort_values(by='id')
    input=input.sort_values(by='id')
    
    nRatings=len(group)
    temp_df=input[input['id'].isin(group['id'].tolist())]
    
    tempRatingList=temp_df['rate'].tolist()
    
    tempGroupList=group['rate'].tolist()
    
    Sxx= sum([i**2 for i in tempRatingList])-pow(sum(tempRatingList),2)/float(nRatings)
    Syy=sum([i**2 for i in tempGroupList])-pow(sum(tempGroupList),2)/float(nRatings)
    Sxy=sum(i*j for i,j in zip(tempRatingList,tempGroupList))-sum(tempRatingList)*sum(tempGroupList)
    
    if Sxx != 0  and Syy !=0:
        pearsonCorrelationDict[name]=Sxy/np.sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name]=0

In [76]:
pearsonCorrelationDict.items()

dict_items([('b', -29.0)])

In [77]:
pearsonDF=pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')

In [78]:
pearsonDF.head()

,0
b,-29.0


In [79]:
pearsonDF.columns=['similarityIndex']
pearsonDF['userid']=pearsonDF.index
pearsonDF.index=range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userid
0,-29.0,b


In [80]:
topUsers=pearsonDF.sort_values(by='similarityIndex',ascending=False)[0:50]
topUsers.head()

,similarityIndex,userid
0,-29.0,b


In [81]:
topUsersRating=topUsers.merge(rating,left_on='userid',right_on='userid',how='inner')
topUsersRating.head()

,similarityIndex,userid,name,rate,id
0,-29.0,b,bun,5,5
1,-29.0,b,ladder,3,6
2,-29.0,b,jump,4,1


In [82]:
topUsersRating['weightRating']=topUsersRating['similarityIndex']*topUsersRating['rate']
topUsersRating.head()

,similarityIndex,userid,name,rate,id,weightRating
0,-29.0,b,bun,5,5,-145.0
1,-29.0,b,ladder,3,6,-87.0
2,-29.0,b,jump,4,1,-116.0


In [83]:
temptopUsersRating=topUsersRating.groupby('id').sum()[['similarityIndex','weightRating']]
temptopUsersRating.columns=['sum_similarityIndex','sum_weightRating']
temptopUsersRating.head()

,sum_similarityIndex,sum_weightRating
id,,
1,-29.0,-116.0
5,-29.0,-145.0
6,-29.0,-87.0


In [84]:
recommendation_df=pd.DataFrame()

recommendation_df['weighted average recommendation score']=temptopUsersRating['sum_weightRating']/temptopUsersRating['sum_similarityIndex']
recommendation_df['id']=temptopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,id
id,,
1,4.0,1
5,5.0,5
6,3.0,6


In [85]:
recommendation_df=recommendation_df.sort_values(by='weighted average recommendation score',ascending=False)
recommendation_df.head()

,weighted average recommendation score,id
id,,
5,5.0,5
1,4.0,1
6,3.0,6


In [86]:
kk=rating.loc[rating['id'].isin(recommendation_df['id'].tolist())]
kk

,userid,name,rate,id
1,b,bun,5,5
5,b,ladder,3,6
6,b,jump,4,1


In [91]:
lst=kk.values.tolist()
lst[0][1]

'bun'